# Library

In [1]:
import pandas as pd
import os
from datetime import datetime

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold

In [70]:
df1_fold = 'D:/data/big2/BD1/df'
an1_fold = 'D:/data/big2/BD1/an'
ods_fold = 'D:/data/big2/ODS'

## 차량 말소 이력

In [106]:
# about 21.9s
name = '말소정보_전처리완료'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3642216 entries, 0 to 3642215
Data columns (total 29 columns):
 #   Column       Dtype  
---  ------       -----  
 0   차량관리번호       object 
 1   본거지법정동코드     int64  
 2   차종_등록정보      object 
 3   차대번호         object 
 4   최초등록일자       int64  
 5   차량연식         int64  
 6   제원관리번호       object 
 7   배출가스인증번호     object 
 8   배출가스등급       object 
 9   차량말소YN_등록정보  object 
 10  차명           object 
 11  연료           object 
 12  등급           object 
 13  차량말소YN_변경이력  object 
 14  변경일자         int64  
 15  변경구분코드       float64
 16  법정동코드        int64  
 17  시도           object 
 18  시구군          object 
 19  읍면동          object 
 20  리            object 
 21  지역3          float64
 22  폐지여부         object 
 23  대기관리권역       object 
 24  대기관리권역YN     object 
 25  변경일자_년       int64  
 26  변경일자_월       int64  
 27  변경일자_일       int64  
 28  변경일자_년_월     float64
dtypes: float64(3), int64(8), object(18)
memory usage: 805.8+ MB


## 조기폐차 이력

In [40]:
# about 0.4s
name = '노후차_조기폐차_관리정보'
file_name = f'{name}.csv'
early = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949')
early.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128567 entries, 0 to 128566
Data columns (total 3 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   VIN                   128567 non-null  object 
 1   ELPDSRC_LAST_APRV_YN  128567 non-null  object 
 2   BSPL_STDG_CD          127286 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.9+ MB


In [41]:
early = early.rename(columns={'VIN':'차대번호', 'ELPDSRC_LAST_APRV_YN':'조기폐차최종승인YN', 'BSPL_STDG_CD':'본거지법정동코드'})
early.columns

Index(['차대번호', '조기폐차최종승인YN', '본거지법정동코드'], dtype='object')

In [42]:
early1 = early[['차대번호', '조기폐차최종승인YN']]
early1.columns

Index(['차대번호', '조기폐차최종승인YN'], dtype='object')

# 전처리

## 필수 컬럼만 추출

In [107]:
df.columns

Index(['차량관리번호', '본거지법정동코드', '차종_등록정보', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '연료', '등급', '차량말소YN_변경이력',
       '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부',
       '대기관리권역', '대기관리권역YN', '변경일자_년', '변경일자_월', '변경일자_일', '변경일자_년_월'],
      dtype='object')

In [13]:
df1 = df[[
    '본거지법정동코드', 
    '차종_등록정보', 
    '차대번호', 
    '최초등록일자', 
    '차량연식', 
    '제원관리번호',
    '배출가스인증번호', 
    '배출가스등급', 
    '차량말소YN_등록정보', 
    '차명', 
    '연료', 
    '등급', 
    '차량말소YN_변경이력',
    '변경일자', 
    '변경구분코드', 
    '법정동코드', 
    '시도', 
    '시구군',
    '변경일자_년', 
    '변경일자_월', 
    '변경일자_일',
    ]]
df1.shape

(3642216, 21)

## 조기폐차여부 정보 추가

In [43]:
df1.shape

(3642216, 21)

In [45]:
df2 = df1.merge(early1, on='차대번호', how='left')
df2.shape

(3642216, 22)

In [47]:
df2[['차대번호', '조기폐차최종승인YN']].head()

,차대번호,조기폐차최종승인YN
0,KMJNG18TPNC005002,NaN
1,KNAFK412BDA020343,NaN
2,KN3HNP6D26K136856,NaN
3,KMFZCS7JP6U149891,NaN
4,KNHMB76136S018476,NaN


In [48]:
df2[['차대번호', '조기폐차최종승인YN']].isnull().sum()

차대번호                0
조기폐차최종승인YN    3537082
dtype: int64

In [49]:
3642216 - 3537082

105134

## 최초등록일자, 변경일자 데이터 타입 확인

In [50]:
df2[['최초등록일자', '변경일자']].dtypes

최초등록일자    int64
변경일자      int64
dtype: object

In [51]:
df2[['최초등록일자', '변경일자']].isnull().sum()

최초등록일자    0
변경일자      0
dtype: int64

In [52]:
df2['최초등록일자'].describe()

count    3642216.00
mean    20059762.63
std        50486.13
min     19551231.00
25%     20030128.00
50%     20051026.00
75%     20090105.00
max     20220628.00
Name: 최초등록일자, dtype: float64

In [53]:
df2['변경일자'].describe()

count    3642216.00
mean    20204827.13
std         9401.69
min     19971111.00
25%     20200306.00
50%     20201130.00
75%     20210818.00
max     20220628.00
Name: 변경일자, dtype: float64

In [55]:
df2[['최초등록일자', '변경일자']].head()

,최초등록일자,변경일자
0,20220628,20220628
1,20121129,20220628
2,20060104,20220628
3,20060105,20220628
4,20060105,20220628


## 차량수명 계산

In [110]:
df3 = df2.copy()

In [111]:
df3['최초등록일자'] = pd.to_datetime(df3['최초등록일자'], format='%Y%m%d')
df3['최초등록일자'].head()

0   2022-06-28
1   2012-11-29
2   2006-01-04
3   2006-01-05
4   2006-01-05
Name: 최초등록일자, dtype: datetime64[ns]

In [112]:
df3['변경일자'] = pd.to_datetime(df3['변경일자'], format='%Y%m%d')
df3['변경일자'].head()

0   2022-06-28
1   2022-06-28
2   2022-06-28
3   2022-06-28
4   2022-06-28
Name: 변경일자, dtype: datetime64[ns]

In [113]:
df3['차량수명_년'] = df3['변경일자'].dt.year - df3['최초등록일자'].dt.year
df3['차량수명_년'].head()

0     0
1    10
2    16
3    16
4    16
Name: 차량수명_년, dtype: int32

In [114]:
df3[['최초등록일자', '변경일자', '차량수명_년']].head()

,최초등록일자,변경일자,차량수명_년
0,2022-06-28,2022-06-28,0
1,2012-11-29,2022-06-28,10
2,2006-01-04,2022-06-28,16
3,2006-01-05,2022-06-28,16
4,2006-01-05,2022-06-28,16


## 차량수명(연식) 계산

In [115]:
df3['차량연식'].isnull().sum()

0

In [116]:
df3['차량수명_년_연식'] = df3['변경일자'].dt.year - df3['차량연식']
df3['차량수명_년_연식'].head()

0     0
1     9
2    16
3    16
4    16
Name: 차량수명_년_연식, dtype: int64

## 연료별 차량수명별 차량 대수

In [117]:
df3['연료'].value_counts(dropna=False)

연료
경유             1660076
휘발유            1528227
LPG(액화석유가스)     412084
휘발유 하이브리드        14497
CNG(압축천연가스)       9204
전기                8018
NaN               7164
LPG 하이브리드         2662
기타연료               118
수소                  99
경유 하이브리드            43
CNG 하이브리드           22
알코올                  1
태양열                  1
Name: count, dtype: int64

In [118]:
df3['조기폐차최종승인YN'].value_counts(dropna=False)

조기폐차최종승인YN
NaN    3537082
Y       105134
Name: count, dtype: int64

### 5등급 경유

In [129]:
df3['등급'].value_counts(dropna=False)

등급
3.0    1198403
5.0    1189945
2.0     783791
4.0     366939
1.0      88793
X        14313
NaN         32
Name: count, dtype: int64

In [131]:
age_col = '차량수명_년'

In [138]:
grp_diesel_early = df3.loc[(df3['연료'] == '경유') & (df3['조기폐차최종승인YN'].isnull() == False) & (df3['등급'] == '5.0')].groupby([age_col], as_index=False)['차대번호'].count()
grp_diesel_early = grp_diesel_early.rename(columns={'차대번호':'조기폐차_차량대수'})
grp_diesel_early

,차량수명_년,조기폐차_차량대수
0,2,2
1,4,3
2,5,5
3,6,1
4,7,4
5,8,6
6,9,3
7,10,2
8,11,4
9,12,5


In [139]:
grp_diesel_early['조기폐차_차량대수'].sum()

105133

In [140]:
grp_diesel = df3.loc[(df3['연료'] == '경유') & (df3['조기폐차최종승인YN'].isnull() == True) & (df3['등급'] == '5.0')].groupby([age_col], as_index=False, dropna=False)['차대번호'].count()
grp_diesel = grp_diesel.rename(columns={'차대번호':'일반_차량대수'})
grp_diesel

,차량수명_년,일반_차량대수
0,0,2
1,1,29
2,2,47
3,3,37
4,4,29
5,5,20
6,6,18
7,7,34
8,8,41
9,9,51


In [141]:
grp_diesel['일반_차량대수'].sum()

1084076

In [142]:
m1 = grp_diesel_early[age_col].max()
m2 = grp_diesel[age_col].max()
if m1 >= m2:
    mx = m1
else:
    mx = m2
mx

59

In [143]:
yd = pd.DataFrame({age_col:[x for x in range(0, mx+1)]})
yd

,차량수명_년
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [144]:
grp = yd.merge(grp_diesel_early, on=age_col, how='left')
grp = grp.merge(grp_diesel, on=age_col, how='left')
grp

,차량수명_년,조기폐차_차량대수,일반_차량대수
0,0,NaN,2.00
1,1,NaN,29.00
2,2,2.00,47.00
3,3,NaN,37.00
4,4,3.00,29.00
5,5,5.00,20.00
6,6,1.00,18.00
7,7,4.00,34.00
8,8,6.00,41.00
9,9,3.00,51.00


In [145]:
grp['조기폐차_차량대수'].max()

18520.0

In [146]:
grp['조기폐차_차량비율'] = grp['조기폐차_차량대수'] / grp['조기폐차_차량대수'].max()
grp['일반_차량비율'] = grp['일반_차량대수'] / grp['일반_차량대수'].max()
grp.head()

,차량수명_년,조기폐차_차량대수,일반_차량대수,조기폐차_차량비율,일반_차량비율
0,0,NaN,2.00,NaN,0.00
1,1,NaN,29.00,NaN,0.00
2,2,2.00,47.00,0.00,0.00
3,3,NaN,37.00,NaN,0.00
4,4,3.00,29.00,0.00,0.00


### [출력] 경유 통계

In [148]:
grp.to_excel(os.path.join(an1_fold, '통계_5등급_경유_차량수명.xlsx'), index=False)

### 휘발유

In [105]:
df3.loc[df3['연료'] == '휘발유', '조기폐차최종승인YN'].value_counts(dropna=False)

조기폐차최종승인YN
NaN    1528227
Name: count, dtype: int64

In [103]:
grp_gasoline_early = df3.loc[(df3['연료'] == '휘발유') & (df3['조기폐차최종승인YN'].isnull() == False)].groupby(['차량수명(년)'], as_index=False)['차대번호'].count()
grp_gasoline_early = grp_gasoline_early.rename(columns={'차대번호':'조기폐차_차량대수'})
grp_gasoline_early

,차량수명(년),조기폐차_차량대수


# 코드 마지막